Download openCV

In [ ]:
!py -m pip uninstall openCV-python-headless
!py -m pip install openCV-python

Test openCV using image

In [1]:
import cv2

image = cv2.imread('image.jpg')
if image is None:
    print("Error: Image not found or path is incorrect")
    exit()

cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Test openCV using webcam

In [1]:
import cv2

cap = cv2.VideoCapture(0)  # 0 for default camera

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Webcam", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Webcam human detection

In [2]:
import cv2

# Initialize video capture (0 for default webcam)
cap = cv2.VideoCapture(0)

# Initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect people in the frame
    (rects, weights) = hog.detectMultiScale(frame, winStride=(8, 8), padding=(16, 16), scale=1.05)

    # Draw bounding boxes
    for (x, y, w, h) in rects:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Webcam Human Detection", frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Leaf detector using green mask (image)

In [ ]:
import cv2
import numpy as np

# 1. Load the image
image = cv2.imread('leaf.png')  # Replace 'leaf.jpg' with your image path
if image is None:
    print("Error: Image not found.")
    exit()

# 2. Convert the image from BGR to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 3. Define range for green color and create a mask
# Adjust these values based on your image's lighting and the shade of green in your leaves
lower_green = np.array([25, 40, 40])
upper_green = np.array([85, 255, 255])
mask = cv2.inRange(hsv, lower_green, upper_green)

# 4. Apply morphological operations to reduce noise
kernel = np.ones((5, 5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# 5. Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 6. Draw the contours on the original image
output = image.copy()
cv2.drawContours(output, contours, -1, (0, 0, 255), 2)

# Optionally, display the number of detected leaves
print("Number of detected leaf regions:", len(contours))

# 7. Display the results
cv2.imshow("Original Image", image)
cv2.imshow("Green Mask", mask)
cv2.imshow("Detected Leaves", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of detected leaf regions: 2


: 

Leaf detector using green mask (webcam)

In [6]:
import cv2
import numpy as np

# Initialize video capture (0 for default webcam)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video capture")
    exit()

# Loop over frames from the camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

    # Convert the frame from BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define range for green color (adjust these values as needed)
    lower_green = np.array([25, 40, 40])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Apply morphological operations to reduce noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours from the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop over the contours and filter/draw them
    for cnt in contours:
        # Optionally filter small contours by area (e.g., area > 500 pixels)
        if cv2.contourArea(cnt) > 500:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            # Optionally, draw the contour itself:
            # cv2.drawContours(frame, [cnt], -1, (255, 0, 0), 2)

    # Display the original frame and the mask (for debugging)
    cv2.imshow("Leaf Detection", frame)
    cv2.imshow("Green Mask", mask)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Detect brownspots on leafs (image)

In [7]:
import cv2
import numpy as np

# 1. Load and preprocess the image
image = cv2.imread('leafFungus.jpg')  # Replace with your image path
if image is None:
    print("Error: Image not found.")
    exit()

# Optional: Resize for faster processing
image = cv2.resize(image, (600, 400))

# Apply a slight blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# 2. Convert image to HSV color space
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

# 3. Define range for brown (dead spots) in HSV
# Note: Adjust these values based on your image and lighting conditions
lower_brown = np.array([10, 50, 20])
upper_brown = np.array([20, 255, 200])
mask = cv2.inRange(hsv, lower_brown, upper_brown)

# 4. Clean up the mask with morphological operations
kernel = np.ones((5, 5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# 5. Detect contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 6. Draw bounding boxes around detected brown/dead spots
output = image.copy()
for cnt in contours:
    if cv2.contourArea(cnt) > 100:  # Filter out small regions; adjust threshold as needed
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(output, (x, y), (x + w, y + h), (0, 0, 255), 2)

# Display the original image, mask, and output
cv2.imshow("Original Image", image)
cv2.imshow("Brown Mask", mask)
cv2.imshow("Detected Dead Spots", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Making the Detection of the leaf better

Scaling the image to only 256x256 for real-time use.

In [ ]:
import cv2
import numpy as np

# 1. Load the image
image = cv2.imread('leafFungus.png')  # Replace 'leaf.png' with your image path
if image is None:
    print("Error: Image not found.")
    exit()

# Scale the image to 256x256
image = cv2.resize(image, (256, 256))

# 2. Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# 3. Convert the image from BGR to HSV
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

# 4. Define range for green color and create a mask
lower_green = np.array([30, 40, 40])  # Adjust these values as needed
upper_green = np.array([90, 255, 255])
mask = cv2.inRange(hsv, lower_green, upper_green)

# 5. Apply morphological operations to reduce noise
kernel = np.ones((7, 7), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# 6. Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 7. Filter contours by area to remove noise
min_area = 500  # Minimum area to consider a contour as a leaf
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

# 8. Draw bounding boxes around each detected leaf
output = image.copy()
for cnt in filtered_contours:
    x, y, w, h = cv2.boundingRect(cnt)  # Get bounding box for each contour
    cv2.rectangle(output, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Draw rectangle

# Optionally, display the number of detected leaves
print("Number of detected leaves:", len(filtered_contours))

# 9. Display the results
cv2.imshow("Original Image", image)
cv2.imshow("Green Mask", mask)
cv2.imshow("Detected Leaves", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

Error: Image not found.


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 